In [7]:
from __future__ import print_function
#import sys
#sys.path.append('../')
import timeit
import h5py
import molml
from molml.features import CoulombMatrix, BagOfBonds
from molml.features import LocalCoulombMatrix
from molml.kernel import AtomKernel
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
import pandas as pd


In [66]:
#load molecule data, the coordinate, atom_number, elements are essential as the input 

attrs_list =[ 'atom_number', 'data_base', 'atom_list', 'atom_coordinate_list', 'frequency_list','target_list',\
          'smile_list', 'atom_charge']

metrics = [r2_score, mean_absolute_error, mean_squared_error, median_absolute_error]

def load_133k(num = None):
    
    data_path = '/home/peng/Documents/Project_C/QSAR_nlp/Dataset_qm9/'
    data_set = "133K.hdf5"

    coord_list = []
    element_list = []
    target_list = []
    atom_no_list = []
    

    f = h5py.File(data_path + data_set, "r")
    if not num:
        limit = len(f)+1
    else:
        limit = num
        
    for i in range(1, limit):
        str_element = str(f[str(i)].attrs[attrs_list[2]], 'utf-8')
        split_str = []
        for j in str_element:
            split_str.append(j)
        element_list.append(np.array(split_str))
        coord_list.append(f[str(i)].attrs[attrs_list[3]])    
        target_list.append(f[str(i)].attrs[attrs_list[-3]]) 
        atom_no_list.append(f[str(i)].attrs[attrs_list[0]])   
        if i %100000 == 0:
            print ("already finish ", i)
    f.close()
    
    #df = pd.DataFrame({'coord':coord_list, 'element':element_list, 'target':target_list})
    
    return element_list, coord_list, target_list, atom_no_list

def get_attributes(element_list, coord_list, target_list):

    fit_list = []
#    target_single_list = []
    for i in range(0, len(element_list)):
        fit_list.append((element_list[i], coord_list[i]))
#        target_single_list.append(target_list[i][-1])
        
    return fit_list #, target_single_list   

In [67]:
def get_train_list(num=None):

    # extract the element, the coordinates, the targets and the atom_no from the 133k dataset
    element_list, coord_list, target_list, atom_no_list = load_133k(num)
    # generate the (element, coordinate) tuple for following feature engineering
    fit_list = get_attributes(element_list, coord_list, target_list)
    return fit_list



In [75]:
# import the input, bob features and targets from csv, and generate train test, train_validation sets
from ast import literal_eval
df = pd.read_csv('bob_targets15.csv', header = 0)

fit_list = get_train_list(num=3)
#df_list_bob = df['bob']

#df_list_newbob = df_list_bob.apply(lambda x : literal_eval(x))

#train_list = (df_list_newbob)



In [76]:
fit_list[0]

(array(['C', 'H', 'H', 'H', 'H'], 
       dtype='<U1'),
 array([[ -1.26981359e-02,   1.08580416e+00,   8.00099580e-03],
        [  2.15041600e-03,  -6.03131760e-03,   1.97612040e-03],
        [  1.01173084e+00,   1.46375116e+00,   2.76574800e-04],
        [ -5.40815069e-01,   1.44752661e+00,  -8.76643715e-01],
        [ -5.23813634e-01,   1.43793264e+00,   9.06397294e-01]]))

In [77]:
# generate feature engineered input features, CM or BOB
feature_methods = [CoulombMatrix(), BagOfBonds()]
feat_co = feature_methods[1]
train_list = feat_co.fit_transform(fit_list)

In [78]:
train_list

array([[ 5.49477531,  5.49476946,  5.49474894,  5.49474169,  0.56081483,
         0.5608061 ,  0.56080599,  0.56080582,  0.56080321,  0.56080291,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.61784735,
         0.61777757,  0.61777591,  0.        ,  0.        ,  0.        ,
         6.88172254,  6.88170334,  6.88158238]])